In [1]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

Current directory: /home/kaveh/Hornsgatan/notebook
Current directory: /home/kaveh/Hornsgatan


In [7]:
import pandas as pd
date = '2020-01-01'
detector = 'w2e_in'
path  = "data/calibration_data/"
number = 100
if number > 0 : 
    postfix = f"{detector}_{date}_{number}"
else:
    postfix = f"{detector}_{date}"


In [9]:
data = pd.read_csv(f'{path}calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_w2e_in,1.577837e+09,8.330000,1.00,1577836811,1.577837e+09,13.8900,9.166667,0.99,-0.836667
1,1_w2e_in,1.577837e+09,8.330000,1.00,1577836814,1.577837e+09,13.8900,7.500000,0.99,0.830000
2,2_w2e_in,1.577837e+09,8.330000,1.00,1577836815,1.577837e+09,13.8900,8.888889,1.76,-0.558889
3,3_w2e_in,1.577837e+09,10.828996,1.30,1577836826,1.577837e+09,18.0570,11.111111,0.09,-0.282115
4,4_w2e_in,1.577837e+09,8.330000,1.00,1577836847,1.577837e+09,13.8900,7.777778,0.99,0.552222
...,...,...,...,...,...,...,...,...,...,...
95,95_w2e_in,1.577838e+09,9.996001,1.20,1577837613,1.577838e+09,16.6680,10.277778,0.10,-0.281777
96,96_w2e_in,1.577838e+09,10.828996,1.30,1577837647,1.577838e+09,18.0570,10.555556,0.09,0.273440
97,97_w2e_in,1.577838e+09,8.330000,1.00,1577837661,1.577838e+09,13.8900,7.777778,0.99,0.552222
98,98_w2e_in,1.577838e+09,8.330000,1.00,1577837663,1.577838e+09,13.8900,7.777778,0.99,0.552222
